In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import ast
import gzip
from collections import defaultdict
import sys

In [2]:
runwayPath = "C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment2\\renttherunway_final_data.json"
print("File path is {}".format(runwayPath))

File path is C:\Users\ramasarma\Documents\UCSD\Fall 2020\CSE 258\Assignment2\renttherunway_final_data.json


In [3]:
## Decomposing fit semantics for product size recommendation in metric spaces
def parseDataFromFile(path):
    """ Parse the data and """
    for l in open(path, "rt", encoding="utf-8"):
        yield ast.literal_eval(l)

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

In [4]:
data = list(parseDataFromFile(runwayPath))
data[10]

{'fit': 'small',
 'user_id': '185966',
 'bust size': '34b',
 'item_id': '1077123',
 'weight': '135lbs',
 'rating': '8',
 'rented for': 'party',
 'review_text': "The dress arrived with a small hole in the beading on the front but wasn't too noticeable. Glad I was able to get two sizes because the 4 was a little tight and would've made for an uncomfortable night of dancing! ",
 'body type': 'athletic',
 'review_summary': "It was fun to wear a dress I wouldn't normally buy! ",
 'category': 'dress',
 'height': '5\' 3"',
 'size': 12,
 'age': '33',
 'review_date': 'January 2, 2018'}

In [5]:
random.seed(1)
random.shuffle(data)

train_data = data[:int(0.8*len(data))]
val_data = data[int(0.8*len(data)):int(0.9*len(data))]
test_data = data[int(0.9*len(data)):]

len(train_data), len(val_data), len(test_data)

(154035, 19254, 19255)

In [6]:
item_data = {}
item_index = {}
user_index = {}
user_data = {}
u_index = 0
i_index = 0
for r in train_data:
    if r['item_id'] + '|' + str(r['size']) not in item_data:
        item_data[r['item_id'] + '|' + str(r['size'])] = [r]
        item_index[r['item_id'] + '|' + str(r['size'])] = i_index
        i_index += 1
    else:
        item_data[r['item_id'] + '|' + str(r['size'])].append(r)
        
    if r['user_id'] not in user_data:
        user_data[r['user_id']] = [r]
        user_index[r['user_id']] = u_index
        u_index += 1
    else:
        user_data[r['user_id']].append(r)

In [7]:
len(user_data), len(user_index), len(item_data), len(item_index)

(90510, 90510, 28938, 28938)

In [8]:
true_size_item = np.zeros(len(item_data))
true_size_cust = np.zeros(len(user_data))
w = 1; b_1 = -1; b_2 = 1; lamda = 2

for item in item_data:
    true_size_item[item_index[item]] = int(item.split('|')[1])

In [9]:
from sklearn.linear_model import LogisticRegression
def calc_auc():
    train_features = []
    train_labels = []
    for r in train_data:
        fe = []
        fe.append(true_size_cust[user_index[r['user_id']]])
        fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
        train_features.append(fe)

        if 'small' in r['fit']:
            train_labels.append(0)
        elif 'fit' in r['fit']:
            train_labels.append(1)
        elif 'large' in r['fit']:
            train_labels.append(2)

    c = 1
    clf_1LV = LogisticRegression(fit_intercept=True, multi_class='ovr', C=c)
    clf_1LV.fit(train_features, train_labels)

    test_features = []; test_labels = []; test_labels_auc = []
    for r in test_data:
        fe = []
        try:
            fe.append(true_size_cust[user_index[r['user_id']]])
        except KeyError:
            fe.append(np.mean(true_size_cust))
        try:
            fe.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]])
        except KeyError:
            fe.append(np.mean(true_size_item))

        test_features.append(fe)
        label = [0, 0, 0]
        if 'small' in r['fit']:
            test_labels.append(0)
            label[0] = 1
        elif 'fit' in r['fit']:
            test_labels.append(1)
            label[1] = 1
        elif 'large' in r['fit']:
            test_labels.append(2)
            label[2] = 1
        test_labels_auc.append(label)

    test_labels_auc = np.array(test_labels_auc)

    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import label_binarize

    pred = clf_1LV.predict_proba(test_features)
    AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)
    
def f(s,t):
    return w*(s-t)

def cal_loss_user(user, cust_size):
    loss = 0
    for r in user_data[user]:
        if 'small' in r['fit']:
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) + b_2)
        elif 'fit' in r['fit']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) - b_2)
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) + b_1)
        elif 'large' in r['fit']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]) - b_1)
    return loss
            
def cal_loss_item(item, product_size):
    loss = 0
    for r in item_data[item]:
        if 'small' in r['fit']:
            loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_2)
        elif 'fit' in r['fit']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_2)
            loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_1)
        elif 'large' in r['fit']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_1)
    return loss

def total_loss():
    loss = 0
    for item in item_data:
        for r in item_data[item]:
            product_size = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]
            if 'small' in r['fit']:
                loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_2)
            elif 'fit' in r['fit']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_2)
                loss += max(0, 1 - f(true_size_cust[user_index[r['user_id']]], product_size) + b_1)
            elif 'large' in r['fit']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['user_id']]], product_size) - b_1)
    return loss

for iterr in range(0,220):
    
    ## Phase 1
    for user in user_data:
        candidate_sizes = []
        for r in user_data[user]:
            if 'small' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_2+1)/w))
            elif 'fit' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_1+1)/w))
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_2-1)/w))
            elif 'large' in r['fit']:
                candidate_sizes.append(true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]] + ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)

        if len(candidate_sizes) == 1:
            true_size_cust[user_index[user]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_user(user, candidate_sizes[s]) - cal_loss_user(user, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_cust[user_index[user]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_cust[user_index[user]] = candidate_sizes[-1]

    ## Phase 2
    for item in item_data:
        candidate_sizes = []
        for r in item_data[item]:
            if 'small' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_2+1)/w))
            elif 'fit' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_1+1)/w))
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_2-1)/w))
            elif 'large' in r['fit']:
                candidate_sizes.append(true_size_cust[user_index[r['user_id']]] - ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)
        if len(candidate_sizes) == 1:
            true_size_item[item_index[item]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_item(item, candidate_sizes[s]) - cal_loss_item(item, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_item[item_index[item]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_item[item_index[item]] = candidate_sizes[-1]

    ## Phase 3
    learning_rate = 0.0000005/np.sqrt(iterr+1)
    grad_w = 0
    grad_b1 = 0
    grad_b2 = 0
    for r in train_data:
        s = true_size_cust[user_index[r['user_id']]]
        t = true_size_item[item_index[r['item_id'] + '|' + str(r['size'])]]

        if 'small' in r['fit']:
            A = 1 - f(s, t) + b_2
            if A>0:
                grad_w += -1*(s - t)
                grad_b2 += 1
        elif 'fit' in r['fit']:
            B = 1 + f(s, t) - b_2
            C = 1 - f(s, t) + b_1
            if B>0:
                grad_w += (s - t)
                grad_b2 += -1
            if C>0:
                grad_w += -1*(s - t)
                grad_b1 += 1
        elif 'large' in r['fit']:
            D = 1 + f(s, t) - b_1
            if D>0:
                grad_w += (s - t)
                grad_b1 += -1

    w -= learning_rate*(grad_w + 2*lamda*w)
    b_1 -= learning_rate*(grad_b1 + 2*lamda*b_1)
    b_2 -= learning_rate*(grad_b2 + 2*lamda*b_2)
    if iterr%5 == 0:
        print(iterr, total_loss())
        calc_auc()

0 121630.86971197347
Average AUC 0.48413346118306105 [0.44669076809140096, 0.5286876255195989, 0.4770219899381833]
5 107117.36507437102
Average AUC 0.4960431820481131 [0.4643772142871304, 0.5205475374949737, 0.5032047943622353]
10 95478.79964351584
Average AUC 0.5000270687601899 [0.4708041823936137, 0.5197436246551159, 0.5095333992318398]
15 87572.24251421353
Average AUC 0.5037419399290989 [0.4743812039644599, 0.5210881968657052, 0.5157564189571315]
20 80183.05220152636
Average AUC 0.5080162708925391 [0.48196408067513885, 0.5203678177246326, 0.5217169142778458]
25 74012.40762371327
Average AUC 0.5116256179819954 [0.48713136826708947, 0.5188119964513354, 0.5289334892275613]
30 68784.73568471683
Average AUC 0.5158699146118032 [0.49372840603936996, 0.5208094100680138, 0.533071927728026]
35 64034.25201989822
Average AUC 0.5213810946540657 [0.5017574869099858, 0.5217035507983445, 0.5406822462538666]
40 60881.22129507508
Average AUC 0.5259886254518197 [0.5073751244429135, 0.5244581090381906,

In [ ]:
# Here, we will compute the latent customer and product sizes for this model
latentCustomerSize = np.zeros(len(userReviews))
latentProductSize = np.zeros(len(itemReviews))

w = 1; b1 = -1; b2 = 1; lamda = 0.1

# We have the latent product size
for item in itemReviews:
    latentProductSize[itemIndex[item]] = int(item.split('|')[1])